```

Sip = [S_x; S_y; S_z];
TCP = [X; Y; Z];

Rx = [1, 0, 0; 0, cos(alpha), sin(alpha); 0, -sin(alpha), cos(alpha)];
Ry = [cos(beta), 0, -sin(beta); 0, 1, 0; sin(beta), 0, cos(beta)];
Rz = [cos(gama), sin(gama), 0; -sin(gama), cos(gama), 0; 0, 0, 1];
R = Rx*Ry*Rz;

v = [v1; v2; v3];
x = [X; Y; Z]-L*R*v;
o = R*v;


a = sqrt(((-R(1,:)*v*TCP(3))/(R(3,:)*v))^2+(-(R(2,:)*v)*TCP(3)/(R(3,:)*v))^2+(-TCP(3))^2);
Q = a*o+TCP;
Sip_proj = Sip+(o'*Q-o'*Sip)*(x-Sip)/(o'*x-o'*Sip);
s_2 = [n1; n2; n3];
s_1 = cross(o,s_2);
k_1 = ((Sip_proj-Q)*s_1)/norm(s_1)^2;
k_2 = ((Sip_proj-Q)*s_2)/norm(s_2)^2;
Q_t = [0; 0];

% Obraz v zobrazovaci rovine
f_1 = (Q_t(1)+k_1*ff/(a+L-ff));
f_2 = Q_t(1)+k_2*ff/(a+L-ff);

% Derivace:
df1_X = diff(f_1, X);
df1_Y = diff(f_1, Y);
df1_Z = diff(f_1, Z);
df1_alpha = diff(f_1, alpha);
df1_beta = diff(f_1, beta);
df1_gama = diff(f_1, gama);
 
df2_X = diff(f_2, X);
df2_Y = diff(f_2, Y);
df2_Z = diff(f_2, Z);
df2_alpha = diff(f_2, alpha);
df2_beta = diff(f_2, beta);
df2_gama = diff(f_2, gama);

```

In [22]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), -sp.sin(alpha)], [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)], [0, 1, 0], [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0], [sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

# Vector v and calculations for x and o
v = sp.Matrix([v1, v2, v3])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.simplify(sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
                        (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z))

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = sp.Matrix([n1, n2, n3])
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2

# Q_t as a zero vector (for 2D image plane coordinates)
Q_t = sp.Matrix([0, 0])

# Calculate image plane coordinates f_1 and f_2
f_1 = Q_t[0] + k_1 * ff / (a + L - ff)
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)
f_both = sp.Matrix([f_1, f_2])

In [25]:


# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_f1 = sp.derive_by_array(f_1, v)


In [54]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1)
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

{x0: sin(beta),
 x1: v3*x0,
 x2: cos(beta),
 x3: cos(gama),
 x4: v1*x3,
 x5: x2*x4,
 x6: sin(gama),
 x7: v2*x6,
 x8: x2*x7,
 x9: -x1 + x5 + x8,
 x10: sin(alpha),
 x11: x10*x3,
 x12: cos(alpha),
 x13: x12*x6,
 x14: x0*x13,
 x15: x11 - x14,
 x16: -v2*x15,
 x17: x10*x6,
 x18: x12*x3,
 x19: x0*x18,
 x20: x17 + x19,
 x21: v1*x20,
 x22: v3*x2,
 x23: x12*x22,
 x24: x21 + x23,
 x25: x16 + x24,
 x26: n1*x25,
 x27: -n3*x9 + x26,
 x28: -Y,
 x29: x10*x22,
 x30: L*x29,
 x31: x0*x17,
 x32: x18 + x31,
 x33: v2*x32,
 x34: x0*x11,
 x35: -x13 + x34,
 x36: v1*x35,
 x37: L*x33 + L*x36 + x28 + x30,
 x38: -S_y - x37,
 x39: -Z,
 x40: L*x16 + L*x21 + L*x23,
 x41: x39 + x40,
 x42: -x41,
 x43: x29 + x33 + x36,
 x44: -x37,
 x45: -X,
 x46: -L*v3*x0 + L*x5 + L*x8 + x45,
 x47: -x46,
 x48: S_x*x9 + S_y*x43 + S_z*x25,
 x49: x25*x42 + x43*x44 + x47*x9 - x48,
 x50: 1/x49,
 x51: x50*x9,
 x52: -v2*x15 + x24,
 x53: sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x52**2),
 x54: x53*x9,
 x55: X + x54,
 x56: x43*x53,
 x57: Y + x56,
 x58: 

In [114]:

import sympy as sp


tmp = reduced_exprs[35][1]

octave_code = sp.printing.octave_code(simplified_expr)
print(octave_code)


{ff.*(x22.*(S_y + x34 - x37 + x39.*(-S_y - x35)) - x27.*(S_z + x32 - x38 + x39.*(-S_z - x33)) + (-n2.*x21 - x28).*(S_x + x29 - x30 + x39.*(-S_x - x31)))./((L - ff + x16).*(x22.^2 + x27.^2 + (n2.*x15 + x28).^2))}


In [26]:
import inspect
gradient_f1_lambd = sp.lambdify(v, gradient_f1, modules='sympy',cse=True)
print(inspect.getsource(gradient_f1_lambd))

def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = sin(beta)
    x1 = v3*x0
    x2 = cos(beta)
    x3 = cos(gama)
    x4 = v1*x3
    x5 = x2*x4
    x6 = sin(gama)
    x7 = v2*x6
    x8 = x1 - x2*x7 + x5
    x9 = sin(alpha)
    x10 = x6*x9
    x11 = cos(alpha)
    x12 = x11*x3
    x13 = -x0*x12 + x10
    x14 = v1*x13
    x15 = x3*x9
    x16 = x11*x6
    x17 = x0*x16 + x15
    x18 = v2*x17
    x19 = v3*x2
    x20 = x11*x19
    x21 = x18 + x20
    x22 = x14 + x21
    x23 = n1*x22 - n3*x8
    x24 = -Y
    x25 = x0*x15 + x16
    x26 = v1*x25
    x27 = x0*x10 - x12
    x28 = -x27
    x29 = v2*x28
    x30 = -L*v3*x2*x9 + L*x26 + L*x29
    x31 = x24 + x30
    x32 = -S_y - x31
    x33 = x19*x9
    x34 = x26 + x29 - x33
    x35 = -x31
    x36 = -Z
    x37 = L*x20
    x38 = L*x14 + L*x18 + x36 + x37
    x39 = -x38
    x40 = -X
    x41 = L*x1
    x42 = -L*v2*x2*x6 + L*x5 + x40 + x41
    x43 = -x42
    x44 = S_x*x8 + S_y*x34 + S_z*x22
    x45 = x22*x39 + x34*x35 + x43*x8 - x44
    x46 = 1

In [95]:
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

{x0: sin(beta),
 x1: v3*x0,
 x2: cos(beta),
 x3: cos(gama),
 x4: v1*x2*x3,
 x5: sin(gama),
 x6: v2*x2*x5,
 x7: -x1 + x4 + x6,
 x8: sin(alpha),
 x9: x3*x8,
 x10: cos(alpha),
 x11: x10*x5,
 x12: x0*x11,
 x13: -x12 + x9,
 x14: -v2*x13,
 x15: x5*x8,
 x16: x10*x3,
 x17: x0*x16,
 x18: v1*(x15 + x17),
 x19: v3*x2,
 x20: x10*x19,
 x21: x18 + x20,
 x22: x14 + x21,
 x23: n1*x22,
 x24: -n3*x7 + x23,
 x25: Abs(x24),
 x26: x19*x8,
 x27: x0*x15,
 x28: x16 + x27,
 x29: v2*x28,
 x30: x0*x9,
 x31: -x11 + x30,
 x32: v1*x31,
 x33: x26 + x29 + x32,
 x34: n1*x33 - n2*x7,
 x35: Abs(x34),
 x36: -v2*x13 + x21,
 x37: -n3*x33,
 x38: n2*x36 + x37,
 x39: Abs(x38),
 x40: x25**2 + x35**2 + x39**2,
 x41: 1/x40,
 x42: -n2*x22 - x37,
 x43: -X,
 x44: sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x36**2),
 x45: x44*x7,
 x46: -L*v3*x0 + L*x4 + L*x6 + x43,
 x47: -S_x - x46,
 x48: -Z,
 x49: L*x14 + L*x18 + L*x20,
 x50: x48 + x49,
 x51: -x50,
 x52: -Y,
 x53: L*x26,
 x54: L*x29 + L*x32 + x52 + x53,
 x55: -x54,
 x56: S_x*x7 + S_y*x33 + S

In [100]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1[3])
reduced_exprs[140][1].subs(subs_dict).subs(subs_dict).subs(subs_dict).subs(
    subs_dict).subs(subs_dict).subs(subs_dict).subs(subs_dict)

(-Derivative(re(sin(gama)*cos(alpha)), alpha) + Derivative(re(sin(alpha)*sin(beta)*cos(gama)), alpha))*re(v1) + (Derivative(re(cos(alpha)*cos(gama)), alpha) + Derivative(re(sin(alpha)*sin(beta)*sin(gama)), alpha))*re(v2) - (-Derivative(im(sin(gama)*cos(alpha)), alpha) + Derivative(im(sin(alpha)*sin(beta)*cos(gama)), alpha))*im(v1) - (Derivative(im(cos(alpha)*cos(gama)), alpha) + Derivative(im(sin(alpha)*sin(beta)*sin(gama)), alpha))*im(v2) + Derivative(re(v3*sin(alpha)*cos(beta)), alpha)

In [72]:
import numpy as np
import inspect
f1_lambd = sp.lambdify(v, f_1, modules=["mpmath"],cse=True)

print(inspect.getsource(f1_lambd))


def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = sin(alpha)
    x1 = cos(gama)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(gama)
    x5 = cos(alpha)
    x6 = x4*x5
    x7 = x2 - x3*x6
    x8 = x0*x4
    x9 = x1*x5
    x10 = v1*(x3*x9 + x8)
    x11 = cos(beta)
    x12 = v3*x11
    x13 = x12*x5
    x14 = x10 + x13
    x15 = -v2*x7 + x14
    x16 = sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x15**2)
    x17 = v1*x1*x11
    x18 = v2*x11*x4
    x19 = -v3*x3 + x17 + x18
    x20 = -v2*x7
    x21 = x14 + x20
    x22 = n1*x21 - n3*x19
    x23 = x0*x12
    x24 = v2*(x3*x8 + x9)
    x25 = v1*(x2*x3 - x6)
    x26 = x23 + x24 + x25
    x27 = n1*x26 - n2*x19
    x28 = -n3*x26
    x29 = -X
    x30 = x16*x19
    x31 = -L*v3*x3 + L*x17 + L*x18 + x29
    x32 = -Z
    x33 = L*x10 + L*x13 + L*x20 + x32
    x34 = -Y
    x35 = L*x23 + L*x24 + L*x25 + x34
    x36 = S_x*x19 + S_y*x26 + S_z*x21
    x37 = x16*x26
    x38 = x16*x21
    x39 = (x19*(X + x30) + x21*(Z + x38) + x26*(Y + x37) - x36)/(-x19*x31 - x

In [ ]:
df1_X = sp.diff(f_1, X)
df1_Y = sp.diff(f_1, Y)
df1_Z = sp.diff(f_1, Z)
df1_alpha = sp.diff(f_1, alpha)
df1_beta = sp.diff(f_1, beta)
df1_gama = sp.diff(f_1, gama)

df2_X = sp.diff(f_2, X)
df2_Y = sp.diff(f_2, Y)
df2_Z = sp.diff(f_2, Z)
df2_alpha = sp.diff(f_2, alpha)
df2_beta = sp.diff(f_2, beta)
df2_gama = sp.diff(f_2, gama)

In [45]:
import sympy as sp

# Define the symbols
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)

# Create the matrix
M = sp.Matrix(5, 3, (S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15))

# Print the matrix
print(M)

Matrix([[S1, S2, S3], [S4, S5, S6], [S7, S8, S9], [S10, S11, S12], [S13, S14, S15]])


In [46]:
# Define the matrix to store the results
results = sp.Matrix(10, 1, [0] * 10)

# Define the loop
for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    
    # Store the results in the matrix
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3
    


In [47]:
import sympy as sp

f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Define the symbols
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))

# Print the vector
print(f_obs)

Matrix([[f_obs1], [f_obs2], [f_obs3], [f_obs4], [f_obs5], [f_obs6], [f_obs7], [f_obs8], [f_obs9], [f_obs10]])


In [52]:
diff = results - f_obs
F_obs = diff.norm()**2

In [49]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, 
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
F_obs_lambd = sp.lambdify(args, F_obs, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

def _lambdifygenerated(alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = sin(beta)
    x1 = cos(gama)
    x2 = cos(beta)
    x3 = x1*x2
    x4 = sin(gama)
    x5 = x2*x4
    x6 = n1*x3 + n2*x5 - n3*x0
    x7 = v3*x0
    x8 = v1*x3
    x9 = v2*x5
    x10 = -L*x7 + L*x8 + L*x9 - X
    x11 = sin(alpha)
    x12 = x1*x11
    x13 = cos(alpha)
    x14 = x13*x4
    x15 = -x0*x14 + x12
    x16 = -v2*x15
    x17 = x11*x4
    x18 = x1*x13
    x19 = x0*x18 + x17
    x20 = v1*x19
    x21 = v3*x2
    x22 = x13*x21
    x23 = x20 + x22
    x24 = x16 + x23
    x25 = L*x16 + L*x20 + L*x22 - Z
    x26 = x11*x21
    x27 = x0*x17 + x18
    x28 = v2*x27
    x29 = x0*x12 - x14
    x30 = v1*x29
    x31 = x26 + x28 + x30
    x32 = L*x26 + L*x28 + L*x30 - Y
    x33 = -x7 + x8 + x9
    x34 = x10*x33 + x24*x25 + x31*x32
    x35 = S13*x33 + S14*x31 + 

In [7]:
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_F_obs = sp.derive_by_array(F_obs, v)

In [8]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
gradient_F_obs_lambd = sp.lambdify(args, gradient_F_obs, modules='numpy', cse=True)

print(inspect.getsource(gradient_F_obs_lambd))

def _lambdifygenerated(alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = sin(gama)
    x1 = cos(alpha)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(alpha)
    x5 = cos(gama)
    x6 = x4*x5
    x7 = -x2 + x3*x6
    x8 = x1*x5
    x9 = x0*x4
    x10 = x3*x9 + x8
    x11 = n2*x10
    x12 = cos(beta)
    x13 = n3*x12
    x14 = x13*x4
    x15 = x11 + x14
    x16 = n1*x7 + x15
    x17 = v3*x12
    x18 = x17*x4
    x19 = L*x18
    x20 = v2*x10
    x21 = v1*x7
    x22 = L*x20 + L*x21 - Y + x19
    x23 = S11 + x22
    x24 = -x2*x3 + x6
    x25 = -x24
    x26 = v2*x25
    x27 = x3*x8 + x9
    x28 = v1*x27
    x29 = x1*x17
    x30 = x28 + x29
    x31 = x26 + x30
    x32 = L*x26 + L*x28 + L*x29
    x33 = -Z + x32
    x34 = -x33
    x35 = x18 + x20
    x36 = x21 + x35
    x37 = -x22
    x38 = v3*x3
    x39 = x12*x5
    x40 = v1*x39
 

In [8]:
import sympy as sp
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v

# Create a 6x6 matrix in sympy
hessian_F_obs = sp.Matrix(6, 6, [0] * 36)

for i in range(6):
    for j in range(6):
        print(f"Done with {i}, {j}")
        hessian_F_obs[i, j] = sp.diff(gradient_F_obs[i], v[j])

Done with 0, 0


/tmp/ipykernel_213808/227576542.py:13: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  hessian_F_obs[i, j] = sp.diff(gradient_F_obs[i], v[j])


Done with 0, 1
Done with 0, 2
Done with 0, 3
Done with 0, 4
Done with 0, 5
Done with 1, 0
Done with 1, 1
Done with 1, 2
Done with 1, 3
Done with 1, 4
Done with 1, 5
Done with 2, 0
Done with 2, 1
Done with 2, 2
Done with 2, 3
Done with 2, 4
Done with 2, 5
Done with 3, 0
Done with 3, 1
Done with 3, 2
Done with 3, 3
Done with 3, 4
Done with 3, 5
Done with 4, 0
Done with 4, 1
Done with 4, 2
Done with 4, 3
Done with 4, 4
Done with 4, 5
Done with 5, 0
Done with 5, 1
Done with 5, 2
Done with 5, 3
Done with 5, 4
Done with 5, 5


In [12]:
print(hessian_F_obs[0,0])

[2000000.0*ff**2*(((n1*(sin(alpha)*sin(gama) + sin(beta)*cos(alpha)*cos(gama)) - n2*(sin(alpha)*cos(gama) - sin(beta)*sin(gama)*cos(alpha)) + n3*cos(alpha)*cos(beta))*(v1*(sin(alpha)*sin(beta)*cos(gama) - sin(gama)*cos(alpha)) + v2*(sin(alpha)*sin(beta)*sin(gama) + cos(alpha)*cos(gama)) + v3*sin(alpha)*cos(beta)) - (n1*(sin(alpha)*sin(beta)*cos(gama) - sin(gama)*cos(alpha)) + n2*(sin(alpha)*sin(beta)*sin(gama) + cos(alpha)*cos(gama)) + n3*sin(alpha)*cos(beta))*(v1*(sin(alpha)*sin(gama) + sin(beta)*cos(alpha)*cos(gama)) - v2*(sin(alpha)*cos(gama) - sin(beta)*sin(gama)*cos(alpha)) + v3*cos(alpha)*cos(beta)))*((-v1*cos(beta)*cos(gama) - v2*sin(gama)*cos(beta) + v3*sin(beta))*(L*v1*cos(beta)*cos(gama) + L*v2*sin(gama)*cos(beta) - L*v3*sin(beta) + S1 - X)/(S1*(v1*cos(beta)*cos(gama) + v2*sin(gama)*cos(beta) - v3*sin(beta)) + S2*(v1*(sin(alpha)*sin(beta)*cos(gama) - sin(gama)*cos(alpha)) + v2*(sin(alpha)*sin(beta)*sin(gama) + cos(alpha)*cos(gama)) + v3*sin(alpha)*cos(beta)) + S3*(v1*(sin(alp

In [13]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
hessian_F_obs_lambd = sp.lambdify(args, hessian_F_obs[3, 0], modules='numpy', cse=True)

print(inspect.getsource(hessian_F_obs_lambd))

def _lambdifygenerated(alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = sin(alpha)
    x1 = cos(gama)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(gama)
    x5 = cos(alpha)
    x6 = x4*x5
    x7 = x2 - x3*x6
    x8 = x0*x4
    x9 = x1*x5
    x10 = x3*x9 + x8
    x11 = n1*x10
    x12 = cos(beta)
    x13 = x12*x5
    x14 = n3*x13
    x15 = x11 + x14
    x16 = -n2*x7 + x15
    x17 = sqrt(v1**2 + v2**2 + v3**2)
    x18 = v1*x10
    x19 = v3*x13
    x20 = x18 + x19
    x21 = -v2*x7 + x20
    x22 = Z/x21
    x23 = sign(x22)
    x24 = x17/x23
    x25 = Z*x24
    x26 = -x7
    x27 = v2*x26
    x28 = L*x18 + L*x19 + L*x27
    x29 = -Z + x28
    x30 = S12 + x29
    x31 = x20 + x27
    x32 = -x29
    x33 = x2*x3 - x6
    x34 = v1*x33
    x35 = x3*x8 + x9
    x36 = v2*x35
    x37 = x0*x12
    x38 = v3*x37
    x39 = x36 + x38
    x4

In [15]:
import numpy as np

res_np = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        xpr_term, xpr_res = sp.cse(hessian_F_obs[i, j])
        res_np[i, j] = len(xpr_term)

res_np

array([[253., 301., 344., 437., 471., 466.],
       [301., 253., 344., 448., 471., 466.],
       [325., 325., 357., 517., 538., 531.],
       [382., 408., 493., 596., 672., 678.],
       [401., 411., 511., 663., 636., 705.],
       [402., 402., 501., 672., 704., 619.]])

In [19]:
list(range(6,6))

[]

In [20]:
for i in range(6):
    for j in range(i+1,6):
        hessian_F_obs[i, j] = 0

In [21]:
xpr_term, xpr_res = sp.cse(hessian_F_obs)

/usr/lib/python3.11/site-packages/sympy/simplify/cse_main.py:865: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  reduced_exprs[i] = Matrix(e.rows, e.cols, reduced_exprs[i])


In [22]:
len(xpr_term)

2737

In [23]:
subs_dict = {}
for var, subexpr in xpr_term:
    subs_dict[var] = subexpr
subs_dict

{x0: sin(gama),
 x1: cos(alpha),
 x2: x0*x1,
 x3: sin(beta),
 x4: sin(alpha),
 x5: cos(gama),
 x6: x4*x5,
 x7: x3*x6,
 x8: -x2 + x7,
 x9: x1*x5,
 x10: x0*x4,
 x11: x10*x3,
 x12: x11 + x9,
 x13: n2*x12,
 x14: cos(beta),
 x15: n3*x14,
 x16: x15*x4,
 x17: x13 + x16,
 x18: n1*x8 + x17,
 x19: v3*x14,
 x20: x19*x4,
 x21: L*x20,
 x22: v2*x12,
 x23: v1*x8,
 x24: L*x22 + L*x23 - Y + x21,
 x25: S11 + x24,
 x26: v3*x3,
 x27: x14*x5,
 x28: v1*x27,
 x29: x0*x14,
 x30: v2*x29,
 x31: x28 + x30,
 x32: -x26 + x31,
 x33: -x32,
 x34: x2*x3,
 x35: -x34 + x6,
 x36: -x35,
 x37: v2*x36,
 x38: x3*x9,
 x39: x10 + x38,
 x40: v1*x39,
 x41: x1*x19,
 x42: x40 + x41,
 x43: x37 + x42,
 x44: L*x41,
 x45: L*x37 + L*x40 + x44,
 x46: -Z + x45,
 x47: -x46,
 x48: x20 + x22,
 x49: x23 + x48,
 x50: -x24,
 x51: L*x28 + L*x30,
 x52: -L*x26 + x51,
 x53: -X + x52,
 x54: -x53,
 x55: -x32*x54 - x43*x47 - x49*x50,
 x56: S10*x32 + S11*x49 + S12*x43,
 x57: x55 + x56,
 x58: 1/x57,
 x59: x33*x58,
 x60: v2*x35,
 x61: x42 - x60,
 x62: 1

In [24]:
subs_dict1 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('Derivative') != -1:
        subs_dict1[var] = subexpr.subs(subs_dict)
subs_dict1

{x261: Derivative(sign(f_obs8 + x103*x111), X),
 x278: Derivative(sign(f_obs10 + x111*x142), X),
 x286: Derivative(sign(f_obs2 + x111*x169), X),
 x294: Derivative(sign(f_obs4 + x111*x196), X),
 x302: Derivative(sign(f_obs6 + x111*x223), X),
 x361: Derivative(sign(f_obs1 + x231*x235), X),
 x367: Derivative(sign(f_obs7 + x235*x242), X),
 x371: Derivative(sign(f_obs9 + x235*x247), X),
 x375: Derivative(sign(f_obs3 + x235*x252), X),
 x379: Derivative(sign(f_obs5 + x235*x257), X),
 x457: Derivative(sign(f_obs8 + x103*x111), Y),
 x458: Derivative(sign(f_obs10 + x111*x142), Y),
 x459: Derivative(sign(f_obs2 + x111*x169), Y),
 x460: Derivative(sign(f_obs4 + x111*x196), Y),
 x461: Derivative(sign(f_obs6 + x111*x223), Y),
 x487: Derivative(sign(f_obs1 + x231*x235), Y),
 x488: Derivative(sign(f_obs7 + x235*x242), Y),
 x489: Derivative(sign(f_obs9 + x235*x247), Y),
 x490: Derivative(sign(f_obs3 + x235*x252), Y),
 x491: Derivative(sign(f_obs5 + x235*x257), Y),
 x689: Derivative(sign(f_obs8 + x103*x

In [25]:
zero_dict = {key: 0 for key in subs_dict1.keys()}
zero_dict

{x261: 0,
 x278: 0,
 x286: 0,
 x294: 0,
 x302: 0,
 x361: 0,
 x367: 0,
 x371: 0,
 x375: 0,
 x379: 0,
 x457: 0,
 x458: 0,
 x459: 0,
 x460: 0,
 x461: 0,
 x487: 0,
 x488: 0,
 x489: 0,
 x490: 0,
 x491: 0,
 x689: 0,
 x690: 0,
 x691: 0,
 x692: 0,
 x693: 0,
 x694: 0,
 x695: 0,
 x696: 0,
 x697: 0,
 x698: 0,
 x699: 0,
 x1162: 0,
 x1163: 0,
 x1164: 0,
 x1165: 0,
 x1166: 0,
 x1168: 0,
 x1169: 0,
 x1170: 0,
 x1171: 0,
 x1172: 0,
 x1181: 0,
 x1837: 0,
 x1838: 0,
 x1839: 0,
 x1840: 0,
 x1841: 0,
 x1843: 0,
 x1844: 0,
 x1845: 0,
 x1846: 0,
 x1847: 0,
 x1852: 0,
 x2592: 0}

In [32]:
# Drop subs_dict entries from subs_dict1.keys()
for key in zero_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(zero_dict)


In [33]:
for key in subs_dict:
    if subs_dict[key] == 0:
        print(key)
        zero_dict[key] = 0


In [34]:
subs_dict2 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('**') != -1:
        if str(subexpr.subs(subs_dict))[0:4] == 'sign':
            subs_dict2[var] = subexpr.subs(subs_dict)
subs_dict2

{x113: sign(f_obs8 + x103*x111)**2,
 x144: sign(f_obs10 + x111*x142)**2,
 x171: sign(f_obs2 + x111*x169)**2,
 x198: sign(f_obs4 + x111*x196)**2,
 x225: sign(f_obs6 + x111*x223)**2,
 x237: sign(f_obs1 + x231*x235)**2,
 x244: sign(f_obs7 + x235*x242)**2,
 x249: sign(f_obs9 + x235*x247)**2,
 x254: sign(f_obs3 + x235*x252)**2,
 x259: sign(f_obs5 + x235*x257)**2,
 x704: sign(x63)**(-2)}

In [35]:
one_dict = {key: 1 for key in subs_dict2.keys()}
one_dict

{x113: 1,
 x144: 1,
 x171: 1,
 x198: 1,
 x225: 1,
 x237: 1,
 x244: 1,
 x249: 1,
 x254: 1,
 x259: 1,
 x704: 1}

In [36]:
# Drop subs_dict entries from subs_dict1.keys()
for key in one_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(one_dict)

In [37]:
for key in subs_dict:
    if subs_dict[key] == 1:
        print(key)
        one_dict[key] = 1

In [38]:
with open('tmp.txt', 'w') as f:
    for key in subs_dict:
        f.write(str(key) + " = " + str(subs_dict[key]) + ";\n")

In [59]:
xpr_res[0][1,1][0]

x118*x484**2 + x145*x480**2 + x1668*x389 + x1669*x390 + x1670*x391 + x1671*x392 + x1672*x393 + x1694*x455 + x1695*x456 + x1696*x457 + x1697*x458 + x1698*x459 + x172*x481**2 + x199*x482**2 + x226*x483**2 + x240*x463**2 + x245*x475**2 + x250*x479**2 + x255*x467**2 + x260*x471**2 + x323*(x1703*x89 + x1704*x82 + x1705*x18) + x332*(x1707*x89 + x1708*x82 + x1709*x18) + x343*(x1710*x89 + x1712*x82 + x1713*x18) + x352*(x1715*x89 + x1716*x82 + x1717*x18) + x361*(x1719*x89 + x1720*x82 + x1721*x18) + x384*(x1710*x229 + x1712*x227 - x1713*x228) + x385*(x1703*x229 + x1704*x227 - x1705*x228) + x386*(x1707*x229 + x1708*x227 - x1709*x228) + x387*(x1715*x229 + x1716*x227 - x1717*x228) + x388*(x1719*x229 + x1720*x227 - x1721*x228)

In [40]:

subs_res = list()
for i in range(6):
    tmp = list()
    for j in range(6):
        try:
            tmp.append(xpr_res[0][i,j][0].subs(zero_dict).subs(one_dict))
        except:
            tmp.append(0)
    subs_res.append(tmp)


In [41]:
with open('tmp.txt', 'a') as f:
    for i in range(6):
        for j in range(6):
            f.write("res(" + str(i+1) + "," + str(j+1) + ") = " + str(subs_res[i][j]) + ";\n")